In [1]:
# jedinka: neko resenje koje kodiram kao niz 0 i 1 - 110010101011

# roditelj1 - 110010101011
# roditelj2 - 001011011100

# 1. nacin da pravimo decu - jedan deo r1 i drugi deo r2 i obrnuto samo spojimo
# dete1 - 110010101100
# dete2 - 001011011011
    
# population = [jedinka1, jedinka2, ...]
# while not stop_condition():
#     izaberi jedinke za reprodukciju
#     ukrstamo jedinke
#     mutacije
#     dobili smo novu generaciju

In [2]:
import random

In [3]:
# UFLP
class Individual:
    def __init__(self, cost, fixedCost):
        numResources = len(fixedCost)
        self.code = [random.random() < 0.25 for _ in range(numResources)]
        self.correctNonFeasible()
        
        self.fitness = self.calcFitness(cost, fixedCost)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def isFeasible(self):
        for c in self.code:
            if c:
                return True
        return False
    
    def invert(self):
        randomResource = random.randrange(len(self.code))
        self.code[randomResource] = not self.code[randomResource]
        
    def correctNonFeasible(self):
        for c in self.code:
            if c:
                return
        randomResource = random.randrange(len(self.code))
        self.code[randomResource] = True
                
        
    def calcFitness(self, cost, fixedCost):
        numUsers = len(cost)
        numResources = len(fixedCost)
        usedResources = [False for _ in range(numResources)]
            
        totalCost = 0
        
        for i in range(numUsers):
            minCost = float('inf')
            resourceUsed = -1
            for j in range(numResources):
                if cost[i][j] < minCost and self.code[i]:
                    minCost = cost[i][j]
                    resourceUsed = j
            totalCost += minCost
            usedResources[resourceUsed] = True
                
        self.code = usedResources
        for j, resource in enumerate(self.code):
            if resource:
                totalCost += fixedCost[j]
            
        fitness = 1 / totalCost
        return fitness

In [4]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for _ in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        return cost, fixedCost

In [5]:
cost, fixedCost = readInput('uflp.txt')

In [6]:
def selection(population):
    TOURNAMENT_SIZE = 5
    bestFitness = -1
    index = -1
    for i in range(TOURNAMENT_SIZE):
        randomIndividual = random.randrange(len(population))
        if population[randomIndividual].fitness > bestFitness:
            bestFitness = population[randomIndividual].fitness
            index = randomIndividual
    return index

In [7]:
def crossover(parent1, parent2, child1, child2):
    breakpoint = random.randrange(len(parent1.code))
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]
    
    child1.code[breakpoint:] = parent2.code[breakpoint:]
    child2.code[breakpoint:] = parent1.code[breakpoint:]

In [8]:
def mutation(individual):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = not individual.code[i]

In [55]:
POPULATION_SIZE = 100
NUM_GENERATIONS = 10
ELITISM_SIZE = POPULATION_SIZE // 5

population = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]
newPopulation = [Individual(cost, fixedCost) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    # elitizam: najboljih nekoliko cemo iskopirati
    population.sort(key=lambda x: x.fitness)
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):
        parent1 = population[selection(population)]
        parent2 = population[selection(population)]

        crossover(parent1, parent2, child1 = newPopulation[j], child2 = newPopulation[j+1])
        mutation(newPopulation[j])
        mutation(newPopulation[j + 1])
        
    population = newPopulation

x = max(population, key=lambda x: x.fitness)
x.code

[False, False, True]